# Training A `TensorFlow.Keras` Classifier With And Without `Horovod`

This test uses MNIST dataset to train a model using TensorFlow.Keras with and without Horovod. Later it will verify that:

  * The accuracy was not damaged in Horovod.
  * The Horovod run was faster (only possible on big data). 

## General Configurations

In [ ]:
# Number of epochs to train (to increase the training time without increasing the memory usage):
N_EPOCHS = 4

# Number of ranks (horovod workers) to deploy for the open mpi job:
N_RANKS = 4

## 1. Training Code

1. Get the MNIST data from `tensorflow.keras.datasets`.
2. Initialize a model.
3. Run training on the training set with validation on the testing set.

Accuracy score will be logged as a result as part of MLRun auto-logging.

In [ ]:
%%writefile func.py
from typing import Tuple
import time
import tensorflow as tf

import mlrun
import mlrun.frameworks.tf_keras as mlrun_tf_keras


def get_datasets(batch_size: int) -> Tuple[tf.data.Dataset, tf.data.Dataset]:
    # Download the data:
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    
    # Initialize tensorflow datasets:
    train_set = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(batch_size)
    test_set = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(batch_size)
    
    return train_set, test_set


def get_model() -> tf.keras.Model:
    # Build the model architecture:
    inputs = tf.keras.Input(shape=(28, 28))
    x = tf.keras.layers.experimental.preprocessing.Rescaling(1.0 / 255)(inputs)
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    x = tf.keras.layers.Dense(128, activation="relu")(x)
    outputs = tf.keras.layers.Dense(10, activation="softmax")(x)
    
    # Initialize a model:
    model = tf.keras.Model(inputs, outputs)
    
    return model


@mlrun.handler(outputs=["time"])
def train(context: mlrun.MLClientCtx, n_epochs: int):
    # Start the timer:
    run_time = time.time()
    
    # Get the data:
    batch_size = 32
    train_set, test_set = get_datasets(batch_size=batch_size)

    # Get the model:
    model = get_model()
    
    # Apply MLRun:
    mlrun_tf_keras.apply_mlrun(model=model, context=context)

    # Compile the model:
    model.compile(
        optimizer=tf.keras.optimizers.legacy.SGD(learning_rate=0.1, momentum=0.9),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )

    # Train:
    model.fit(
        train_set,
        validation_data=test_set,
        epochs=n_epochs,
        steps_per_epoch=len(train_set) // batch_size,
    )
    run_time = time.time() - run_time
    
    return run_time

## 2. Create a Project

1. Create the MLRun project.
2. Create an MLRun function of the training code.

In [ ]:
import numpy as np
import mlrun

In [ ]:
# Create the project:
project = mlrun.get_or_create_project(name="horovod-tensorflow-test", context="./", user_project=True)

In [ ]:
# Create the job function:
# TODO: bump to 2.18.1 after ML-10271 fix
job_function = project.set_function("./func.py", name="train_job", kind="job", image="mlrun/mlrun", handler="train", requirements=["tensorflow==2.15.1"])
job_function.apply(mlrun.auto_mount())
job_function.deploy()

In [ ]:
project.build_image(image=f'.tfimage',
                    base_image='mlrun/mlrun',
                    requirements=['tensorflow==2.15.1'],
                    overwrite_build_params=True,
                    set_as_default=False)

In [ ]:
N_RANKS = 4  # adjust to number of workers you want

mpijob_function = project.set_function(
    "./func.py",
    name="train_mpijob",
    kind="mpijob",
    image=".tfimage",
    handler="train",
)

mpijob_function.apply(mlrun.auto_mount())
mpijob_function.spec.replicas = N_RANKS

builder_env = {
    'HOROVOD_WITH_MPI': '1',
    'HOROVOD_WITH_TENSORFLOW': '1',
    'HOROVOD_WITHOUT_MLX': '1',
    'HOROVOD_WITHOUT_PYTORCH': '1',
    'HOROVOD_WITHOUT_MXNET': '1',
    'HOROVOD_WITHOUT_GLOO': '1',
    'HOROVOD_WITHOUT_XLA': '1',
}

commands = [
    'apt update --fix-missing \
    && apt upgrade -y \
    && apt install -y \
         llvm-dev \
         build-essential \
         cmake \
         gcc-12 g++-12 \
    && update-alternatives --install /usr/bin/gcc gcc /usr/bin/gcc-12 60 \
    && update-alternatives --install /usr/bin/g++ g++ /usr/bin/g++-12 60 \
    && update-alternatives --set gcc /usr/bin/gcc-12 \
    && update-alternatives --set g++ /usr/bin/g++-12 \
    && apt clean \
    && rm -rf /var/lib/apt/lists/*',
    'pip install horovod==0.28.1',
    'pip install typing_extensions==4.14.1'
]


mpijob_function.spec.build.commands=commands
mpijob_function.deploy(builder_env=builder_env)

## 3. Run As A Job

Run the training as a `job` and storing the results.

In [ ]:
# Run as a job:
job_run = job_function.run(
    name="training_job",
    output_path="/tmp/",
    params={
        "n_epochs": N_EPOCHS,
    },
)

# Store results:
job_time = job_run.status.results['time']
job_accuracy = job_run.status.results['validation_accuracy']

## 4. Run As a MPIJob

Run the training as a `mpijob` and storing the results.

In [ ]:
# Run as a mpijob:
mpijob_run = mpijob_function.run(
    name="training_mpijob",
    output_path="/tmp/",
    params={
        "n_epochs": N_EPOCHS,
    },
)

# Store results:
mpijob_time = mpijob_run.status.results['time']
mpijob_accuracy = mpijob_run.status.results['validation_accuracy']

## 5. Compare Runtimes

1. Print a summary message.
2. Verify that:
  * The mpijob run took less time (only in stronger machines). 
  * The accuracy value is equal between the runs.

In [ ]:
# Delete the MLRun project:
mlrun.get_run_db().delete_project(name=project.name, deletion_strategy="cascading")

In [ ]:
# Print the test's collected results:
print(
    f"Job:\n" 
    f"\t{'%.2f' % job_time} Seconds\n"
    f"\tAccuracy: {job_accuracy}"
)
print(
    f"Open MPI Job (Horovod):\n"
    f"\t{'%.2f' % mpijob_time} Seconds\n"
    f"\tAccuracy: {mpijob_accuracy}\n"
)

# Verification: (Only possible to test on a stronger machine (requires big data and longer training)
# assert mpijob_time < job_time
# assert np.isclose(job_accuracy, mpijob_accuracy, atol=0.1)